## SVM Model

### Library and Data Importation

In [2]:
#import sys
#!{sys.executable} -m pip install cvxpy
import cvxpy as cp
import pandas as pd
import numpy as np
from numpy import array
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

#import data set
nfl1 = pd.read_csv("NFL_data_super_cleaned.csv")

nfl1.dropna(inplace=True)
display(nfl1)
print(nfl1.isnull().values.any(), nfl1.isnull().sum().sum())

,posteam,yardline_100,quarter_seconds_remaining,qtr,down,goal_to_go,ydstogo,play_type,score_margin
0,PIT,58.0,893.0,1,1.0,0.0,10,pass,0.0
1,PIT,53.0,856.0,1,2.0,0.0,5,run,0.0
2,PIT,56.0,815.0,1,3.0,0.0,8,pass,0.0
3,PIT,56.0,807.0,1,4.0,0.0,8,kick,0.0
4,TEN,98.0,796.0,1,1.0,0.0,10,run,0.0
...,...,...,...,...,...,...,...,...,...
353055,CAR,71.0,82.0,4,2.0,0.0,1,pass,-5.0
353056,CAR,71.0,77.0,4,3.0,0.0,1,pass,-5.0
353057,CAR,66.0,63.0,4,2.0,0.0,10,pass,-5.0
353058,CAR,66.0,58.0,4,3.0,0.0,10,pass,-5.0


False 0


### Data Preprocessing

In [4]:
#Our 350,000 samples seem like a little too much, so sample about 10,000 rows
sample = nfl1.sample(n=100000, random_state=21, axis=0)
print(sample.isnull().values.any(), sample.isnull().sum().sum())
display(sample)
#one-hot encode the categorical variables
#posteam_type, defteam, side_of_field, game_date (drop), time (convert?), yrdline (convert?)

cat_columns = ["posteam" , "qtr"]
#one-hot encode categorical variables
encoder = preprocessing.OneHotEncoder()
cat_array = encoder.fit_transform(sample[cat_columns]).toarray()
cat_labels = encoder.get_feature_names_out(cat_columns)
cat_onehot_encoded = pd.DataFrame(cat_array, columns=cat_labels)

#Add back the continuous variables
cat_onehot_encoded["yardline_100"] = sample["yardline_100"]
cat_onehot_encoded["quarter_seconds_remaining"] = sample["quarter_seconds_remaining"]
cat_onehot_encoded["down"] = sample["down"]
cat_onehot_encoded["goal_to_go"] = sample["goal_to_go"]
cat_onehot_encoded["ydstogo"] = sample["ydstogo"]
cat_onehot_encoded["score_margin"] = sample["score_margin"]


cat_onehot_encoded["play_type"] = sample["play_type"]
cat_onehot_encoded.dropna(inplace=True)


display(cat_onehot_encoded)
print(cat_onehot_encoded.isnull().values.any(), cat_onehot_encoded.isnull().sum().sum())
#print(onehot_encoded.isnull().values.any())
#split data into training and testing sets
#seed: 21, train/test ratio: 0.2 test, 0.8 train



x, y = cat_onehot_encoded.drop(["play_type"], axis=1), cat_onehot_encoded["play_type"]
#Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 21)



False 0


,posteam,yardline_100,quarter_seconds_remaining,qtr,down,goal_to_go,ydstogo,play_type,score_margin
222356,SEA,61.0,119.0,1,3.0,0.0,22,run,0.0
61840,CAR,78.0,702.0,1,4.0,0.0,4,kick,-7.0
2684,HOU,61.0,585.0,4,2.0,0.0,4,run,0.0
225580,JAX,73.0,483.0,3,3.0,0.0,3,pass,-3.0
46520,NO,70.0,201.0,1,1.0,0.0,10,run,7.0
...,...,...,...,...,...,...,...,...,...
111341,CIN,41.0,330.0,2,2.0,0.0,15,pass,8.0
50695,MIN,57.0,59.0,4,1.0,0.0,10,pass,-10.0
84887,HOU,13.0,828.0,4,1.0,0.0,10,run,7.0
89944,SEA,16.0,448.0,1,2.0,0.0,7,run,7.0


,posteam_ARI,posteam_ATL,posteam_BAL,posteam_BUF,posteam_CAR,posteam_CHI,posteam_CIN,posteam_CLE,posteam_DAL,posteam_DEN,...,qtr_3,qtr_4,qtr_5,yardline_100,quarter_seconds_remaining,down,goal_to_go,ydstogo,score_margin,play_type
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,53.0,856.0,2.0,0.0,5.0,0.0,run
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,56.0,815.0,3.0,0.0,8.0,0.0,pass
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,94.0,731.0,3.0,0.0,6.0,0.0,run
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,43.0,684.0,1.0,0.0,10.0,0.0,pass
14,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,89.0,500.0,1.0,0.0,10.0,0.0,run
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,30.0,316.0,4.0,0.0,7.0,17.0,pass
99993,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,71.0,261.0,2.0,0.0,8.0,-23.0,pass
99994,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,37.0,254.0,1.0,0.0,10.0,23.0,run
99996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,37.0,166.0,3.0,0.0,10.0,23.0,run


False 0


### Multi-class SVM Model Implementation

In [5]:
#compares each play_type to the other possible play_type

scaler = preprocessing.StandardScaler()
scaler.fit(X_train)

clf_ovo = SVC(kernel='linear', decision_function_shape='ovo') # The other is ovr

clf_ovo.fit(scaler.transform(X_train), np.asarray(y_train))

print(classification_report(y_test, clf_ovo.predict(scaler.transform(X_test))))

              precision    recall  f1-score   support

        kick       0.88      0.98      0.93       558
        pass       0.70      0.70      0.70      2954
         run       0.59      0.58      0.59      2147

    accuracy                           0.68      5659
   macro avg       0.73      0.75      0.74      5659
weighted avg       0.68      0.68      0.68      5659

